# Markov Movie Recommender Program


In [72]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip

--2020-12-08 18:10:22--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip     31%[=====>              ]  48.39M  5.39MB/s    eta 21s    ^C
unziping ...
Archive:  moviedataset.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of moviedataset.zip or
        moviedataset.zip.zip, and cannot find m

In [74]:
import pandas as pd
from math import sqrt
import numpy as np

In [263]:
#Putting movie data and ratings data from csv into a Pandas DataFrame
moviesDataFrame = pd.read_csv("movies.csv")
ratingsDataFrame = pd.read_csv("ratings.csv")
#Displaying first 5 rows of the movies dataframe
moviesDataFrame.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [264]:
#Start preproccessing and changing the dataframe to fit the program's specifications
moviesDataFrame['year'] = moviesDataFrame.title.str.extract('(\(\d\d\d\d\))',expand=False)
moviesDataFrame['year'] = moviesDataFrame.year.str.extract('(\d\d\d\d)',expand=False)
moviesDataFrame['title'] = moviesDataFrame.title.str.replace('(\(\d\d\d\d\))', '')
moviesDataFrame['title'] = moviesDataFrame['title'].apply(lambda x: x.strip())
moviesDataFrame.head()
#Removed year from title and made into a separate coulumn

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [265]:
#Make the Genres into a list in the dataframe
moviesDataFrame["genres"] = moviesDataFrame.genres.str.split('|')
moviesDataFrame.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [266]:
#Start making the dataframe that relates movies to the genres
#Find all the genres that apply to each movie with a 1 and those that don't with a 0
moviesGenresDataFrame = moviesDataFrame.copy()

for i, r in moviesDataFrame.iterrows():
    for g in r["genres"]:
        moviesGenresDataFrame.at[i, g] = 1

moviesGenresDataFrame = moviesGenresDataFrame.fillna(0)
moviesGenresDataFrame.head()


,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [267]:
#Start the second characteristic of movie recommending: ratings
ratingsDataFrame.head()
#Take out time from the dataframe
ratingsDataFrame = ratingsDataFrame.drop("timestamp", 1)
ratingsDataFrame.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [268]:
#Take in a user's input of movies that he or she has watched or likes with the ratings of the movies to act as keys
#Add as many movies as the user wants
userInput = [{'title':'Transformers', 'rating':3.5}, {'title':'Toy Story', 'rating':3.5}, {'title':'Jumanji', 'rating':2}, {'title':"Pirates of the Caribbean: On Stranger Tides", 'rating':3.3}, {'title':'Interstellar', 'rating':4.3}]
# userInput = [{'title':'Tuxedo, The', 'rating':3.1}, {'title':'Rush Hour 3', 'rating':3.1}, {'title':'Dragon Blade', 'rating':2.5}, {'title':"Medallion, The", 'rating':3.5}, {'title':'Shanghai Noon', 'rating':3.3}]

#Put the input into a dataframe 
inputMovies = pd.DataFrame(userInput)

#Find movie ID from the movieDataframe that correlates to the User's movies
inputId = moviesDataFrame[moviesDataFrame['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
#Exclude the genres and year from this list
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,53996,Transformers,3.5
3,86880,Pirates of the Caribbean: On Stranger Tides,3.3
4,109487,Interstellar,4.3


In [269]:
#Begin creating the transition matrix which will include a combination of ratings and genres for the movies 

#Start with genres
moviesUserDataFrame = moviesGenresDataFrame[moviesGenresDataFrame["movieId"].isin(inputMovies["movieId"].tolist())]
moviesUserDataFrame


,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11984,53996,Transformers,"[Action, Sci-Fi, Thriller, IMAX]",2007,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
17223,86880,Pirates of the Caribbean: On Stranger Tides,"[Action, Adventure, Fantasy, IMAX]",2011,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
23044,109487,Interstellar,"[Sci-Fi, IMAX]",2014,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [270]:
#Clean up to only include the genres for the movies
moviesUserDataFrame = moviesUserDataFrame.reset_index(drop=True)
genreUserDataFrame = moviesUserDataFrame.drop("movieId", 1).drop("title", 1).drop("genres", 1).drop("year", 1)
genreUserDataFrame

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [271]:
#Start "learning" about the user by combining the ratings and genre values above
profileUserDataFrame = genreUserDataFrame.transpose().dot(inputMovies["rating"])
profileUserDataFrame
#This is the combination of genre/ratings calcuated from the input matrix. It will be used with the transition matrix to create a recommendation

Adventure              8.8
Animation              3.5
Children               5.5
Comedy                 3.5
Fantasy                8.8
Romance                0.0
Drama                  0.0
Action                 6.8
Crime                  0.0
Thriller               3.5
Horror                 0.0
Mystery                0.0
Sci-Fi                 7.8
IMAX                  11.1
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [272]:
#Begin to create transition matrix 

#Create genre quantitative table from the MovieLens entire dataset
genreTableDataFrame = moviesGenresDataFrame.set_index(moviesGenresDataFrame["movieId"])
genreTableDataFrame = genreTableDataFrame.drop("movieId", 1).drop("title", 1).drop("genres", 1).drop("year", 1)
genreTableDataFrame.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [273]:
#Start the recommendation by multiplying the table above with the profile and then take the weighted average
recommendationDataFrame = ((genreTableDataFrame*profileUserDataFrame).sum(axis=1))/(profileUserDataFrame.sum())
recommendationDataFrame.head()

movieId
1    0.507589
2    0.389545
3    0.059022
4    0.059022
5    0.059022
dtype: float64

In [274]:
#Begin to clean the recommendations by sorting them from highest to lowest
recommendationDataFrame = recommendationDataFrame.sort_values(ascending=False)
recommendationDataFrame.head()

movieId
32031     0.826307
85261     0.792580
103042    0.730185
54278     0.694772
47124     0.694772
dtype: float64

In [275]:
#Finally display the final recommendation table by using the movieId as a key to search the movie dataset
finalRecommendation = moviesDataFrame.loc[moviesDataFrame["movieId"].isin(recommendationDataFrame.head(50).keys())]
finalRecommendation

,movieId,title,genres,year
542,546,Super Mario Bros.,"[Action, Adventure, Children, Comedy, Fantasy,...",1993
664,673,Space Jam,"[Adventure, Animation, Children, Comedy, Fanta...",1996
4686,4781,Megiddo: The Omega Code 2,"[Action, Adventure, Fantasy, Sci-Fi, Thriller]",2001
5784,5882,Treasure Planet,"[Adventure, Animation, Children, Sci-Fi, IMAX]",2002
6252,6350,Laputa: Castle in the Sky (Tenkû no shiro Rapy...,"[Action, Adventure, Animation, Children, Fanta...",1986
6261,6365,"Matrix Reloaded, The","[Action, Adventure, Sci-Fi, Thriller, IMAX]",2003
6823,6934,"Matrix Revolutions, The","[Action, Adventure, Sci-Fi, Thriller, IMAX]",2003
8283,8965,"Polar Express, The","[Adventure, Animation, Children, Fantasy, IMAX]",2004
8783,26340,"Twelve Tasks of Asterix, The (Les douze travau...","[Action, Adventure, Animation, Children, Comed...",1976
8942,26590,G.I. Joe: The Movie,"[Action, Adventure, Animation, Children, Fanta...",1987


In [278]:
#Check to see if extra movie user likes is on the top 50
if "Journey to the Center of the Earth" in finalRecommendation["title"].values:
    print("success")
else:
    print("no")

success
